## Deploying GPT-OSS Models with SMJS and SageMaker Inference Components

This notebook demonstrates how to deploy OpenAI’s GPT‑OSS models (20B) on Amazon SageMaker JumpStart using **inference components**. Inference components let you host multiple copies of a model—or even different models—on a single endpoint and route traffic to specific components.

**SM Inference Components**

![IC](imgs/sm-mme-01.png)

These models provide a 128 k token context window and support configurable reasoning levels (low, medium, or high). To enable web search grounding, disable network isolation and supply an EXA API key via the `EXA_API_KEY` environment variable when deploying the model.

**gpt-oss-20b in SMJS console**

![IC](imgs/smjs-img.png)

In the deployment example below we provision a single `p5.48xlarge` instance (since this is the SMJS default) and attach two 20B inference components using one GPU each (tensor parallel degree = 1). You can route inference to a particular component by specifying its `InferenceComponentName` when you invoke the endpoint.


In [39]:
import sagemaker, boto3
from sagemaker.jumpstart.model import JumpStartModel
from sagemaker.compute_resource_requirements.resource_requirements import ResourceRequirements

sess = sagemaker.Session()
smrt = boto3.client("sagemaker-runtime")

endpoint_name = sagemaker.utils.name_from_base("oai-oss-mix")

role = sagemaker.get_execution_role()

#### Prerequisites

You can either execute this notebook in SageMaker Studio or locally. If you are new to Amazon SageMaker, please follow the guidance provided here: [Guide to getting set up with Amazon SageMaker
](https://docs.aws.amazon.com/sagemaker/latest/dg/gs.html). 

For local setup please make sure to configure configure the [AWS CLI](https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-configure.html) with your credentials, install necessary libraries and ensure you have [permissions for SageMaker](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html).

In addition to either setting up your local environment with the local permissions or setting up SageMaker Studio please also make sure to check instance quota. This Jupyter Notebook itself can be run on a ml.t3.medium instance. To deploy the model to the SageMaker endpoint, you may need to request a quota increase. To request a quota increase, follow these steps:

1. Navigate to the Service Quotas console.
2. Choose Amazon SageMaker.
3. Review your default quota for the following resources:
    - ml.p5.48xlarge for endpoint usage
4. If needed, request a quota increase for these resources.
5. By default, models in SageMaker JumpStart run in network isolation. The GPT OSS models come with a built-in tool for web search using EXA, a meaning-based web search API powered by embeddings. To use this tool, OpenAI requires customers get an [API key from EXA](https://exa.ai/?gad_source=1&gad_campaignid=22502546132&gbraid=0AAAAA-z0DiAZu9OD-gshUnVtITyrtBazd&gclid=CjwKCAjw49vEBhAVEiwADnMbbJJ0-UITnd31rsQ1gyGsCQA5LWWcQuY5y6hbtFpjAMOz82wlqdGCcxoCCQsQAvD_BwE) and pass this key as an environment variable to their JumpStartModel instance when deploying it through the SageMaker Python SDK. 

Let's spin up a an endpoint with two model copies at `tp=1`. For context, a `p5.48xlarge` has 8 H100s so you're able to go up to 8 model copies per instance for `gpt-oss-20b`

In [41]:
# ---------------------------
# Inference Component: 20B (2 copies @ TP=1)
# ---------------------------
ic_20b_name  = f"ic-20b-{endpoint_name}"
model_20b_id = "openai-reasoning-gpt-oss-20b"      # JumpStart id (OAI GPT-OSS 20B)

env_20b = {
    "OPTION_TENSOR_PARALLEL_DEGREE": "1",
    "EXA_API_KEY": "<insert your Exa api key>"
}

js_20b = JumpStartModel(
    model_id=model_20b_id,
    model_version=model_version,
    enable_network_isolation=False,
    name=f"{endpoint_name}-20b",
    env=env_20b,
)

_ = js_20b.deploy(
    accept_eula=True,
    instance_type="ml.p5.48xlarge",                 # same endpoint/instance
    initial_instance_count=1,
    container_startup_health_check_timeout=900,
    endpoint_name=endpoint_name,                    # attach to existing endpoint
    endpoint_type=sagemaker.enums.EndpointType.INFERENCE_COMPONENT_BASED,
    inference_component_name=ic_20b_name,
    resources=ResourceRequirements(
        requests={
            "num_accelerators": 1,  # per-copy GPU reservation
            "memory": 8192,
            "copies": 2             # two 20B copies; SageMaker LB across them
        }
    ),
)


Using model 'openai-reasoning-gpt-oss-20b' with wildcard version identifier '*'. You can pin to version '1.0.0' for more stable results. Note that models may have different input/output signatures after a major version upgrade.
No instance type selected for inference hosting endpoint. Defaulting to ml.p5.48xlarge.
INFO:sagemaker.jumpstart:No instance type selected for inference hosting endpoint. Defaulting to ml.p5.48xlarge.
INFO:sagemaker:Creating model with name: oai-oss-mix-2025-08-08-15-55-27-742-20b
INFO:sagemaker:Creating inference component with name ic-20b-oai-oss-mix-2025-08-08-15-55-27-742 for endpoint oai-oss-mix-2025-08-08-15-55-27-742


--------------------------------!

In [50]:
print("Endpoint:", endpoint_name)
print("IC 20B:", ic_20b_name)

Endpoint: oai-oss-mix-2025-08-08-15-55-27-742
IC 20B: ic-20b-oai-oss-mix-2025-08-08-15-55-27-742
IC 20B : ic-20b-oai-oss-mix-2025-08-08-15-55-27-742


## Harmony response format and inference examples

When interacting with GPT‑OSS models you must use the **Harmony response format**. Each message has a **role** and may specify a **channel**. Harmony supports five roles:

| Role | Purpose |
|-----|---------|
| **system** | Specifies the model’s identity, knowledge cutoff, current date, reasoning level, available channels and built‑in tools. |
| **developer** | Provides instructions for the model (similar to a system prompt) and declares available function tools. |
| **user** | Represents the end‑user’s input. |
| **assistant** | Represents the model’s output; may be a tool call or a message. |
| **tool** | Contains the output returned by an external tool; the tool name itself is used as the role. |

Assistant messages can be tagged with one of three channels:

| Channel | Purpose |
|-------|---------|
| **analysis** | Holds the model’s chain‑of‑thought reasoning. These messages are **not** intended for end‑users. |
| **commentary** | Used for function/tool calls and sometimes for preambles before calling multiple tools. |
| **final** | Contains the user‑visible answer or output. |

### Example inference payloads

The example code in this notebook demonstrates a simple function call flow:

1. Compose a single string that concatenates the system message, the developer message (with tool definitions), the user’s question and an `Assistant:` marker.
2. Define the same function in the `tools` array of the payload.
3. Invoke the model. It responds with a `function_call` specifying the tool name and arguments on the commentary channel.
4. Execute the tool on the client, format its JSON result, and send it back in a tool message (role =`tool`).
5. Invoke the model again with the appended tool message. It replies on the final channel with a human‑readable answer.

After an assistant message in the final channel you should drop any previous chain‑of‑thought content when forming the next request. If the last message was a tool call, include both the chain‑of‑thought and the tool call before you continue the conversation.

In the following code cells we route inference to the `ic‑20b` inference component and demonstrate the EXA‑powered `web_search_preview` tool (enabled via the `EXA_API_KEY` in the environment) to answer questions with citations.


The **System Message** is to specify built-in tools, available channels, etc. It is different from the system prompt. The built-in tools are specified in typescript format. Here we use the built-in web search tool with **EXA**.

In [65]:
from datetime import date

CURRENT_DATE = date.today().isoformat()

SYSTEM_CORE = (
    "You are ChatGPT, a large language model trained by OpenAI.\n"
    "Knowledge cutoff: 2024-06\n"
    f"Current date: {CURRENT_DATE}\n\n"
    "reasoning: medium\n\n"
    "# Valid channels: analysis, commentary, final. Channel must be included for every message.\n"
    "Calls to these tools must go to the commentary channel: 'functions'."
)

# Built-in browser tool 
BROWSER_BLOCK = """
# Tools
## browser
// Tool for browsing.
// The `cursor` appears in brackets before each browsing display: `[{cursor}]`.
// Cite information from the tool using the following format:
// `【{cursor}†L{line_start}(-L{line_end})?】`.
// Do not quote more than 10 words directly from the tool output.
// sources=web (default: web)
namespace browser {
// Searches for information related to `query` and displays `topn` results.
type search = (_: {
query: string,
topn?: number, // default: 10
source?: string,
}) => any;
// Opens the link `id` from the page indicated by `cursor`.
type open = (_: {
id?: number | string, // default: -1
cursor?: number, // default: -1
loc?: number, // default: -1
num_lines?: number, // default: -1
view_source?: boolean, // default: false
source?: string,
}) => any;
// Finds exact matches of `pattern` in the current page, or the page given by `cursor`.
type find = (_: {
pattern: string,
cursor?: number, // default: -1
}) => any;
} // namespace browser
"""
def build_system_message(include_browser=True, include_python=False) -> str:
    parts = [SYSTEM_CORE]
    if include_browser:
        parts.append(BROWSER_BLOCK)
    return "\n".join(parts)

The **Developer Message** is where you specify the function calls and system prompt. Functions are typically in typescript.

All functions that are available to the model should be defined in the developer message in a dedicated Tools section.
To define the functions we use a TypeScript-like type syntax and wrap the functions into a dedicated functions namespace. It’s important to stick to this format closely to improve accuracy of function calling. You can check out the harmony renderer codebase for more information on how we are turning JSON schema definitions for the arguments into this format but some general formatting practices:

* Define every function as a type {function_name} = () => any if it does not receive any arguments
* For functions that receive an argument name the argument _ and inline the type definition
* Add comments for descriptions in the line above the field definition
* Always use any as the return type
* Keep an empty line after each function definition
* Wrap your functions into a namespace, generally functions is the namespace you should use to not conflict with other tools that the model might have been trained on.



In [66]:
DEVELOPER_INSTRUCTIONS = "You are a helpful AI assistant. Provide clear, concise, and helpful responses."

FUNCTIONS_BLOCK = """
# Tools
## functions
namespace functions {
// Gets the current weather for a specific location.
type get_current_weather = (_: {
// The city and state/country, e.g. "San Francisco, CA" or "London, UK"
location: string,
// Temperature unit preference
unit?: "celsius" | "fahrenheit", // default: celsius
}) => any;
} // namespace functions
"""

def build_developer_message(include_functions=True) -> str:
    body = ["# Instructions", DEVELOPER_INSTRUCTIONS]
    if include_functions:
        body.append(FUNCTIONS_BLOCK)
    return "\n\n".join(body)


In [108]:
def build_input(system_msg: str, developer_msg: str, user_text: str) -> str:
    return (
        f"System: {system_msg.strip()}\n\n"
        f"Developer: {developer_msg.strip()}\n\n"
        f"Human: {user_text.strip()}\n\n"
        "Assistant:"
    )

SYSTEM_MSG = build_system_message(include_browser=True)
DEVELOPER_MSG = build_developer_message(include_functions=True)

INPUT_STR_WEATHER_TOOL = build_input(
    SYSTEM_MSG,
    DEVELOPER_MSG,
    "What is the weather like in Seattle?"
)

INPUT_STR_BROWSER_TOOL = build_input(
    SYSTEM_MSG,
    DEVELOPER_MSG,
    "Who is the current President of the US? Use Browser tool."
)


#### Weather function call example

The following cell sends a Harmony-formatted request to the 20B inference component using the `get_current_weather` tool.

We build a single-string prompt that includes the system and developer messages, declare the tool in the `tools` array, and ask about the weather. The model will respond with a tool call in the commentary channel, specifying the location and unit. We print the raw JSON response to inspect the Harmony structure.

In [109]:
import json

TOOLS_ARRAY = [
    {
      "type": "function",
      "name": "get_current_weather",
      "description": "Gets the current weather for a specific location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The city and state/country, e.g. 'San Francisco, CA' or 'London, UK'"
          },
          "unit": {
            "type": "string",
            "enum": ["celsius", "fahrenheit"],
            "default": "celsius",
            "description": "Temperature unit preference"
          }
        },
        "required": ["location"]
      }
    }
]

payload_weather = {
  "model": "/opt/ml/model",
  "input": INPUT_STR_WEATHER_TOOL,                                 
  "instructions": DEVELOPER_INSTRUCTIONS,             
  "max_output_tokens": 2048,
  "stream": "false",
  "temperature": 0.7,
  "reasoning": {"effort": "medium"},
  "tool_choice": "auto",
  "tools": TOOLS_ARRAY
}

payload_browser = {
  "model": "/opt/ml/model",
  "input": INPUT_STR_BROWSER_TOOL,                                
  "instructions": DEVELOPER_INSTRUCTIONS,             
  "max_output_tokens": 2048,
  "stream": "false",
  "temperature": 0.7,
  "reasoning": {"effort": "medium"},
  "tool_choice": "auto",
  "tools": TOOLS_ARRAY
}

In [110]:
response = smrt.invoke_endpoint(
    EndpointName="oai-oss-mix-2025-08-08-15-55-27-742",
    InferenceComponentName="ic-20b-oai-oss-mix-2025-08-08-15-55-27-742",  # <- set to route to a specific IC if you want
    ContentType="application/json",
    Accept="application/json",
    Body=json.dumps(payload_weather),
)

print(response["Body"].read().decode("utf-8"))

{"id":"resp_70830f03b78847d8aa20d4520b0a759d","created_at":1754779573,"instructions":"You are a helpful AI assistant. Provide clear, concise, and helpful responses.","metadata":null,"model":"/opt/ml/model","object":"response","output":[{"id":"rs_3ef4c581c73e4672a7e7b6695b00401c","content":[{"text":"The user asks: \"What is the weather like in Seattle?\" We have a function to get current weather. So we need to call it.","type":"reasoning_text"}],"summary":[],"type":"reasoning","encrypted_content":null,"status":null},{"arguments":"{\"location\":\"Seattle, WA\",\"unit\":\"celsius\"}","call_id":"call_b363c9d6206c4fec9bfa0789b1186ff6","name":"get_current_weather","type":"function_call","id":"ft_b363c9d6206c4fec9bfa0789b1186ff6","status":null}],"parallel_tool_calls":true,"temperature":0.7,"tool_choice":"auto","tools":[{"name":"get_current_weather","parameters":{"type":"object","properties":{"location":{"type":"string","description":"The city and state/country, e.g. 'San Francisco, CA' or 'Lo

#### Browser use example

The following cell sends a Harmony-formatted request to the 20B inference component and prompts the use of the built-in web search tool.

If the model decides to call actions in the browser it will use the same format as for function calls with two notable exceptions:

1. Requests will be made to the analysis channel
2. The recipient will be `browser.search`, `browser.open`, `browser.find` respectively



In [113]:
response = smrt.invoke_endpoint(
    EndpointName="oai-oss-mix-2025-08-08-15-55-27-742",
    InferenceComponentName="ic-20b-oai-oss-mix-2025-08-08-15-55-27-742",  # <- set to route to a specific IC if you want
    ContentType="application/json",
    Accept="application/json",
    Body=json.dumps(payload_browser),
)

print(response["Body"].read().decode("utf-8"))

{"id":"resp_7ee668a090154baa8863b3421d6e4275","created_at":1754779604,"instructions":"You are a helpful AI assistant. Provide clear, concise, and helpful responses.","metadata":null,"model":"/opt/ml/model","object":"response","output":[{"id":"rs_a99d3df16e934a2bab0f835e0acfb2c4","content":[{"text":"The user asks: \"Who is the current President of the US? Use Browser tool.\" They want us to browse and provide a current answer. The current date is 2025-08-09. So we need to verify that the current President is Joe Biden? Wait, Joe Biden was elected in 2020, inaugurated 2021. But the next election is 2024, and likely the next president might be someone else. As of 2025, it's likely Joe Biden still is president? Let's check. The term for a US president is 4 years. Biden started 20 Jan 2021. His term ended 20 Jan 2025. If he didn't run, someone else could be president. In 2024, the winner was Donald Trump or someone else. Actually as of 2025, the current president might be someone else. Let'

---

### Conclusion

The outputs above illustrate how GPT‑OSS models use the Harmony format to separate reasoning, tool usage, and final answers.

In the first invocation, the assistant produces reasoning in the analysis channel, then emits a tool call on the commentary channel. In the second invocation, a built-in web search is triggered via the EXA‑powered tool, and citations are included in the final response.

This concludes our demonstration of deploying GPT‑OSS models on SageMaker, invoking them with Harmony‑compliant prompts, and handling function calls.